In [2]:
import os
if "COLAB_GPU" in os.environ:
  print("Running in Google Collab")
  !pip install PyMuPDF # For reading the PDF's
  !pip install tqdm #Make fast for loops and iterations
  !pip install accelerate
  !pip install bitsandbytes
  !pip install flash-attn --no-build-isolation


Running in Google Collab
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 60.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [3]:
!pip install -y torch torchvision torchaudio transformers sentence-transformers
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -U transformers sentence-transformers


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -y
Looking in indexes: https://download.pytorch.org/whl/cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.0.1
    Uninstalling sentence-transformers-3.0.1:
      Successfully uninstalled sentence-transformers-3.0.1


Document or pdf processing


In [4]:
import os
import requests
#get PDF Document
pdf_path = "human-nutrition-text.pdf"
# Downloading if PDF is not exist in your computer
if not os.path.exists(pdf_path):
  print("File doesnt exist, dowmloading..")
  #URL of PDF to download
  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"
  file_name = pdf_path # saving the pdf locally
  # send a get request
  response = requests.get(url)
  # check if request was succesful
  if response.status_code == 200:
    with open(file_name, 'wb') as file:
      file.write(response.content)
    print(f"file saved {file_name}")
  else:
    print(f"Failed: {response.staus_code}")
else:
  print(f"File {pdf_path} exists:")


File doesnt exist, dowmloading..
file saved human-nutrition-text.pdf


In [5]:
import fitz # fitz is slightly better than PymuPDF in this case
from tqdm.auto import tqdm # For Speed Progress
def text_formatter(text: str) ->str:
  cleaned_text = text.replace("\n"," ").strip() # Cleaning up the white spaces
  return cleaned_text
def open_and_read_pdf(pdf_path: str) -> list[dict]:
  # Opens a pdf files and reads every line
  # Pdf_path is used because it needs to find the pdf
  doc = fitz.open(pdf_path) # opening the pdf
  pages_and_text = []
  for page_number,page in tqdm(enumerate(doc)):
     # iterating the document
     text = page.get_text() # getting the text
     cleaned_text = text_formatter(text) # cleaning the text
     pages_and_text.append({"page_number": page_number-41,
                            "page_text": cleaned_text, # Added page_text key
                            "Page_char_count ": len(cleaned_text) ,"page_word_count ":len(cleaned_text.split(" ")), "page_sentence_count_raw ":len(cleaned_text.split(". ")), "page_token_count ": len(cleaned_text)/4})
  return pages_and_text
pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_text': 'Human Nutrition: 2020 Edition',
  'Page_char_count ': 29,
  'page_word_count ': 4,
  'page_sentence_count_raw ': 1,
  'page_token_count ': 7.25},
 {'page_number': -40,
  'page_text': '',
  'Page_char_count ': 0,
  'page_word_count ': 1,
  'page_sentence_count_raw ': 1,
  'page_token_count ': 0.0}]

In [6]:
import random
random.sample(pages_and_texts,k=3)

[{'page_number': 1022,
  'page_text': 'Image by  USDA / CC  BY 4.0  How Will I Know if My Food Has Been  Irradiated?  The FDA requires that irradiated foods bear the international  symbol for irradiation. Look for the Radura symbol along with the  statement “Treated with radiation” or “Treated by irradiation” on  the food label. Bulk foods, such as fruits and vegetables, are required  to be individually labeled or to have a label next to the sale  container. The FDA does not require that individual ingredients in  multi-ingredient foods (e.g., spices) be labeled. It is important to  remember that irradiation is not a replacement for proper food  handling practices by producers, processors, and consumers.  Irradiated foods need to be stored, handled, and cooked in the  same way as non-irradiated foods, because they could still become  contaminated with disease-causing organisms after irradiation if  the rules of basic food safety are not followed.1  1.\xa0Food Irradiation. (2018). U.S. 

In [9]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,Page_char_count,page_word_count,page_sentence_count_raw,page_token_count
0,-41,31,5,1,7.75
1,-40,0,1,1,0.00
2,-39,322,43,1,80.50
3,-38,214,31,2,53.50
4,-37,799,115,2,199.75


In [7]:
df.info()

NameError: name 'df' is not defined

Now we will check the 5 chuncking strategies
1.Fixed_Size
2.Semantic
3.Structural
4.Recursive
5.LLM Based

In [ ]:
def chunk_text(text: str, chunk_size: 100) ->list:
  chunks =[]
  current_chunk = ''
  words = text.split()
  for word in words:
    if len(current_chunk) + len(word) + 1 <=chunk_size: # We are checking our chunk is greater than chunk_size
      current_chunk += (word+' ')
    else:
      chunks.append(current_chunk.strip()) # if it is greater then we create a new chunk
      current_chunk = word + ' ' # Added a space here
  if current_chunk:
    chunks.append(current_chunk.strip())
  return chunks


def chunk_pdf_pages(pages_and_texts: list, chunk_size: 500) ->list[dict]:
  all_chunks = []
  for page in pages_and_texts:
    page_number = page["page_number"]
    page_text = page["page_text"] # Corrected key to page_text
    chunks = chunk_text(page_text, chunk_size=chunk_size)
    for i, chunk in enumerate(chunks):
      all_chunks.append({
          "page_number": page_number,
          "chunk_number": i,

          "chunk_text": chunk
      })
  return all_chunks
chunked_pages = chunk_pdf_pages(pages_and_texts, chunk_size = 500)
print(f"Total Chunks: {len(chunked_pages)}")

In [1]:
!pip -q install --upgrade "sentence-transformers==3.0.1" "transformers<5,>=4.41" scikit-learn nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.2 MB/s eta 0:00:00


In [10]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
nltk.download('punkt',quiet='True')
#Loading
semantic_model = SentenceTransformer("all-MiniLM-L6-v2")
def semantic_chunk(text: str, similarity_threshold: float = 0.8, max_tokens: int = 500 ) ->list:
  sentences = nltk.sent_tokenize(text)
  if not sentences:
    return []
  embeddings = semantic_model.encode(sample_data)
  chunks = []
  current_chunk = [sentences[0]]
  current_embedding = embeddings[0]
  for i in range(1, len(sentences)):
    sim = cosine_similarity([current_embedding], embeddings[i])[0][0]
    chunk_token_count = len(" ".join(current_chunk)) // 4
    if sim >= similarity_threshold and chunk_token_count < max_tokens:
